In [1]:
import torch
import pandas as pd
import numpy as np
import time

In [2]:
df = pd.read_csv('gmc_data.csv')

In [3]:
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,qual
0,0.572,0.532,0,-6.844,1,0.0316,0.214,0.000000,0.2840,0.607,76.033,spotify:track:6rZwsNzGS7nq8qILqdM9kX,252267,4,1
1,0.565,0.437,6,-7.680,0,0.0403,0.194,0.000007,0.0656,0.217,96.132,spotify:track:5P3yUXUC9rZPJPNmYGKEAz,236733,3,1
2,0.587,0.525,9,-9.257,0,0.1980,0.099,0.000250,0.5570,0.323,90.356,spotify:track:3ZGrUoFhNCWmm6sbFAuktX,241320,4,1
3,0.443,0.157,7,-12.818,1,0.0382,0.757,0.000000,0.1130,0.401,139.953,spotify:track:4JuYMhZk1SfhImtaHO3Sh6,194680,3,1
4,0.537,0.890,5,-5.191,1,0.0488,0.117,0.000021,0.2790,0.597,114.088,spotify:track:2Zd9awmsA8KuF65snhz7YH,231387,4,1


In [4]:
df.shape

(200, 15)

In [5]:
#  df[df['qual'] == 1]
# np.zeros(df.size[0])
df['bad'] = pd.Series(np.zeros(df.shape[0], dtype=int))
df.loc[100:200, 'bad'] = 1

In [6]:
labels = df[['qual', 'bad']].values
labels.shape

(200, 2)

In [7]:
# get rid of the URI column
df = df[['danceability', 'energy', 'loudness', 
    'speechiness', 'acousticness', 'instrumentalness', 
    'liveness', 'valence', 'tempo', 'duration_ms',
    'time_signature']].values
df -= np.mean(df, axis=0)
df.shape

(200, 11)

In [8]:
N, D_in, H, D_out = 200, 11, 10, 2

In [9]:
torch.cuda.is_available()

True

In [31]:
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda")

In [37]:
# x = torch.randn(N, D_in, device=device, dtype=dtype)
x = torch.as_tensor(data=df, device=device, dtype=dtype)
y = torch.as_tensor(data=labels, device=device, dtype=dtype)
# x.size()

In [38]:
x.device

device(type='cpu')

In [39]:
x[0]

tensor([-2.6560e-02, -1.1322e-01,  1.0462e-01, -4.8549e-02, -1.3800e-02,
        -3.0143e-02,  1.1328e-01,  1.3635e-01, -4.5181e+01,  4.0619e+04,
         9.0000e-02])

In [45]:
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate=1e-13

In [49]:
# gpu took 40.919s
# cpu took 18.169s
# ????????
start = time.time()
for t in range(1):
    # Forward pass - will be different every time
    # y_pred is a torch.tensor on cuda.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
#     print(y_pred)
    
    
    # Compute loss - will be different every time 
    # loss is a torch.tensor on cuda
    loss = (y_pred - y).pow(2).sum()

#     if t % 10000 == 0:
#         print(y_pred)
#         print("loss at iteration ", t , " : " , loss.item())
    
    # Backprop to compute gradients - autograd will compute the gradient of loss
    # with resepct to all tensors with requires_grad=True.
    # w1.grad and w2.grad will be tensors holding the gradient of the loss wrt to w1 w2
    loss.backward()
    # print(w1.grad)
    
    # Update the weights using gradient descent. wrap in torch.no_grad() because 
    # weights have requires_grad=True; but we don't need to track this. 
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # set the gradients back to zero 
        w1.grad.zero_()
        w2.grad.zero_()
print('Final Loss: ', loss.item(), '. Took ', time.time() - start, ' seconds to run.')

Final Loss:  835738009600.0 . Took  0.0007927417755126953  seconds to run.
